In [16]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec

In [30]:
namespace = utils.get_default_target_namespace()

model_name='torch2'
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

In [31]:
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=model_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                               pytorch=(V1beta1SKLearnSpec(
                                   storage_uri="pvc://user1-volume/kserve-pytorch"))))
)

In [32]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2023-04-06T06:39:56Z',
  'generation': 1,
  'labels': {'serviceEnvelope': 'kserve'},
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {}, 'f:pytorch': {'.': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-04-06T06:39:54Z'}],
  'name': 'torch2',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '671394',
  'uid': '1e72d499-feeb-434e-a86e-a7d2e89afe1a'},
 'spec': {'predictor': {'model': {'modelFormat': {'name': 'pytorch'},
    'name': '',
    'resources': {},
    'runtime': 'kserve-torchserve',
    'storageUri': 'pvc://user1-volume/kserve-pytorch'}}}}

In [33]:
KServe.get(model_name, namespace=namespace)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2023-04-06T06:39:56Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'labels': {'serviceEnvelope': 'kserve'},
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {}, 'f:pytorch': {'.': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-04-06T06:39:54Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:finalizers': {'.': {},
       'v:"inferenceservice.finalizers"': {}}}},
    'manager': 'manager',
    'operation': 'Update',
    'time': '2023-04-06T06:39:56Z'},
   {'apiVersion': 'serving.kse

In [34]:
import requests

isvc_resp = KServe.get(model_name, namespace=namespace)
isvc_url = isvc_resp['status']['address']['url']

print(isvc_url)

inference_input = {
  'instances': [
    [6.8,  2.8,  4.8,  1.4],
    [6.0,  3.4,  4.5,  1.6]
  ]
}

response = requests.post(isvc_url, json=inference_input)
print(response.text)

http://torch2.kubeflow-user-example-com.svc.cluster.local/v2/models/torch2/infer
{"predictions": [{"output": ["Iris-versicolor", "Iris-versicolor"]}]}


In [48]:
requests.post(isvc_url, json=inference_input)

<Response [500]>

In [37]:
import requests

isvc_url = "http://torch-iris.kubeflow-user-example-com.svc.cluster.local/v2/models/torch-iris/infer"
"http://192.168.0.5:8080/v1/models/my_torch_iris:predict"
print(isvc_url)

inference_input = {
  'instances': [
    [6.8,  2.8,  4.8,  1.4],
    [6.0,  3.4,  4.5,  1.6]
  ]
}

response = requests.post(isvc_url, json=inference_input)
print(response.text)

http://torch-iris.kubeflow-user-example-com.svc.cluster.local/v2/models/torch-iris/infer
<html><title>404: Model with name torch-iris does not exist.</title><body>404: Model with name torch-iris does not exist.</body></html>
